In [8]:
!pip install nltk
import nltk
nltk.download('punkt_tab')


Defaulting to user installation because normal site-packages is not writeable


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/itbecomesteam/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [9]:
!pip install transformers
!pip install accelerate
!pip install hf_xet
!pip install datasets
!pip install bert-score
!pip install blobfile tiktoken
!pip install torch==2.3.1 --index-url https://download.pytorch.org/whl/cu121
!pip install datasets==3.6.0
!pip install tokenizers==0.14.1 
!pip install huggingface_hub==0.32.0
!pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
INFO: pip is looking at multiple versions of tokenizers to determine which version is compatible with other requirements. This could take a while.
  Using cached tokenizers-0.15.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.15.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.14.1
    Uninstalling tokenizers-0.14.1:
      Successfully uninstalled tokenizers-0.14.1
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defau

In [11]:
import shutil
shutil.rmtree("/root/.cache/huggingface/metrics", ignore_errors=True)

In [13]:
!pip install --upgrade torch --index-url https://download.pytorch.org/whl/cu121


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.5.1%2Bcu121-cp312-cp312-linux_x86_64.whl (780.4 MB)
  Using cached https://download.pytorch.org/whl/cu121/nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
  Using cached https://download.pytorch.org/whl/nvidia_nccl_cu12-2.21.5-py3-none-manylinux2014_x86_64.whl (188.7 MB)
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.20.5
    Uninstalling nvidia-nccl-cu12-2.20.5:
      Successfully uninstalled nvidia-nccl-cu12-2.20.5
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 8.9.2.26
    Uninstalling nvidia-cudnn-cu12-8.9.2.26:
      Successfully uninstalled nvidia-cudnn-cu12-8.9.2.26
  Attempting uninstall: torch
    Found existing installation: torch 2.3.1+cu121
    Uninstalling torch-2.

In [ ]:
import os
import gc
import math
import pandas as pd
import numpy as np
import torch
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
)
from evaluate import load
from nltk.tokenize import word_tokenize
import editdistance
import torch.nn.functional as F
import psutil
from tqdm import tqdm
from torch.utils.data import DataLoader



os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"

#  하드웨어 정보 출력 및 캐시 정리
def show_hardware_status():
    print("\n[Hardware Info]")
    if torch.cuda.is_available():
        print("CUDA Available:", torch.cuda.is_available())
        print("Device Name:", torch.cuda.get_device_name(0))
        print("Memory Allocated (MB):", torch.cuda.memory_allocated(0) / 1024 ** 2)
        print("Memory Cached (MB):", torch.cuda.memory_reserved(0) / 1024 ** 2)
        print("Total Memory (MB):", torch.cuda.get_device_properties(0).total_memory / 1024 ** 2)
    else:
        print("CUDA not available. Using CPU.")
    print("CPU Cores:", psutil.cpu_count(logical=False))
    print("Logical CPUs:", psutil.cpu_count(logical=True))
    print("Total RAM (GB):", round(psutil.virtual_memory().total / 1024**3, 2))
    print("Cleaning GPU Cache...\n")
    gc.collect()
    torch.cuda.empty_cache()

show_hardware_status()

# Model/tokenizer load (base or small)
model_name = "google/mt5-small"  # mT5-small/mt5-base" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# T5 계열 학습 시 필수 설정
model.config.use_cache = False
model.config.decoder_start_token_id = tokenizer.eos_token_id

# Full-data loading

raw_df = pd.read_csv("/home/itbecomesteam/yai/Data_filtered.csv", encoding="utf-8")
# raw_df = raw_df[["source", "ko"]].rename(columns={"source": "input_text", "ko": "target_text"})
raw_df = raw_df[["ko_translationese", "ko"]].rename(columns={"ko_translationese": "input_text", "ko": "target_text"})
raw_df = raw_df.dropna()
raw_df["input_text"] = raw_df["input_text"].astype(str)
raw_df["target_text"] = raw_df["target_text"].astype(str)

raw_df = raw_df[~raw_df["input_text"].str.strip().eq("")]
raw_df = raw_df[~raw_df["target_text"].str.strip().eq("")]

raw_dataset = Dataset.from_pandas(raw_df)
dataset_split = raw_dataset.train_test_split(test_size=0.2, seed=42)
full_ds = DatasetDict({
    "train": dataset_split["train"],
    "validation": dataset_split["test"]
})

# Preprocessing
def preprocess(example):
    model_inputs = tokenizer(
        example["input_text"], max_length=128, padding="max_length", truncation=True
    )
    labels = tokenizer(
        text_target=example["target_text"], max_length=128, padding="max_length", truncation=True
    )["input_ids"]
    labels = [label if label != tokenizer.pad_token_id else -100 for label in labels]
    model_inputs["labels"] = labels
    return model_inputs

full_tokenized = full_ds.map(preprocess, remove_columns=full_ds["train"].column_names)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)

# 4. 정확한 perplexity 계산 방식 (문장별 토큰수 반영)
val_loader = DataLoader(
    full_tokenized["validation"],
    batch_size=8,
    collate_fn=data_collator
)

from torch.utils.data import DataLoader


def compute_perplexity_from_preds(preds, refs):
    total_logp = 0.0
    total_tokens = 0

    for pred_text, ref_text in zip(preds, refs):
        input_ids = tokenizer(pred_text, return_tensors="pt", padding=True, truncation=True).input_ids.to(model.device)
        labels = tokenizer(ref_text, return_tensors="pt", padding=True, truncation=True).input_ids.to(model.device)
        with torch.no_grad():
            output = model(input_ids=input_ids, labels=labels)
        loss = output.loss
        n_tokens = (labels != -100).sum().item()
        total_logp += loss.item() * n_tokens
        total_tokens += n_tokens

    return math.exp(total_logp / total_tokens) if total_tokens > 0 else float("inf")


# Matric
def compute_metrics(eval_preds):
    pred_ids = eval_preds.predictions
    label_ids = eval_preds.label_ids

    # padding 토큰 처리
    pred_ids = np.where(pred_ids != -100, pred_ids, tokenizer.pad_token_id)
    label_ids = np.where(label_ids != -100, label_ids, tokenizer.pad_token_id)

    # 디코딩
    preds = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    refs = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    preds = [p.strip() for p in preds]
    refs = [r.strip() for r in refs]

    preds_nonempty = [p for p in preds if p]
    refs_nonempty = [r for r in refs if r]

    if not preds_nonempty or not refs_nonempty:
        return {
            "MATTR": 0.0,
            "KoBERTScore_F1": 0.0,
            "Perplexity": float("inf"),
            "Levenshtein_Distance": float("inf")
        }

    # MATTR
    def mattr(texts, window_size=50):
        def single(text):
            tokens = text.split()
            if len(tokens) < window_size:
                return len(set(tokens)) / len(tokens) if tokens else 0.0
            return np.mean([
                len(set(tokens[i:i+window_size])) / window_size
                for i in range(len(tokens) - window_size + 1)
            ])
        return np.mean([single(t) for t in texts])

    mattr_score = mattr(preds_nonempty)

    # BERTScore (KoBERT)
    bs_metric = load("bertscore", keep_in_memory=True)
    bs_results = bs_metric.compute(predictions=preds_nonempty, references=refs_nonempty, lang="ko")
    f1_kobert = float(np.mean(bs_results["f1"]))

    # Levenshtein Distance
    import editdistance
    ld = np.mean([editdistance.eval(p, r) for p, r in zip(preds_nonempty, refs_nonempty)])

    # PPL: preds + refs 
    def compute_perplexity_from_preds(preds, refs):
        model.eval()
        total_logp = 0.0
        total_tokens = 0

        for pred_text, ref_text in zip(preds, refs):
            input_ids = tokenizer(pred_text, return_tensors="pt", padding=True, truncation=True).input_ids.to(model.device)
            labels = tokenizer(ref_text, return_tensors="pt", padding=True, truncation=True).input_ids.to(model.device)
            labels[labels == tokenizer.pad_token_id] = -100
            with torch.no_grad():
                outputs = model(input_ids=input_ids, labels=labels)
                loss = outputs.loss
            n_tokens = (labels != -100).sum().item()
            total_logp += loss.item() * n_tokens
            total_tokens += n_tokens

        return math.exp(total_logp / total_tokens) if total_tokens > 0 else float("inf")

    ppl = compute_perplexity_from_preds(preds_nonempty, refs_nonempty)

    return {
        "MATTR": round(mattr_score, 4),
        "KoBERTScore_F1": round(f1_kobert, 4),
        "Perplexity": round(ppl, 4),
        "Levenshtein_Distance": round(ld, 2),
    }



✅ [Hardware Info]
CUDA Available: True
Device Name: NVIDIA RTX A6000
Memory Allocated (MB): 2227.1845703125
Memory Cached (MB): 2290.0
Total Memory (MB): 48550.4375
CPU Cores: 20
Logical CPUs: 40
Total RAM (GB): 188.54
Cleaning GPU Cache...



Map:   0%|          | 0/85593 [00:00<?, ? examples/s]

Map:   0%|          | 0/21399 [00:00<?, ? examples/s]

In [15]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=-100 
)



training_args = Seq2SeqTrainingArguments(
    output_dir="output/mt5_small-translationese-mitigation",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=5,
    weight_decay=0.01,
    predict_with_generate=True,
    generation_max_length=64,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=True,  # 가능하면 fp16=True 로 설정
    report_to=["wandb", "tensorboard"]
)



trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=full_tokenized["train"],
    eval_dataset=full_tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/home/itbecomesteam/.local/lib/python3.12/site-packages/accelerate/accelerator.py:449: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [16]:
import torch
print(torch.__version__)  #  최소 2.2 이상이어야 함


2.5.1+cu121


In [17]:
trainer.train()

wandb: Currently logged in as: cosmic4dev (cosmic4dev-yonsei-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/home/itbecomesteam/.local/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Mattr,Kobertscore F1,Perplexity,Levenshtein Distance
1,2.608600,2.058083,0.992200,0.853800,9.215000,15.590000
2,2.336800,1.941612,0.993500,0.856700,7.968300,15.250000
3,2.247700,1.874399,0.993900,0.858000,7.392200,15.100000
4,2.193000,1.846628,0.993500,0.858700,7.138100,15.050000
5,2.214000,1.839588,0.993600,0.858800,7.098200,15.030000


/home/itbecomesteam/.local/lib/python3.12/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Checkpoint destination directory output/mt5_small-translationese-mitigation/checkpoint-2675 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/home/itbecomesteam/.local/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/itbecomesteam/.local/lib/python3.12/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated

TrainOutput(global_step=13375, training_loss=2.6290146393107476, metrics={'train_runtime': 18914.4208, 'train_samples_per_second': 22.626, 'train_steps_per_second': 0.707, 'total_flos': 5.65715903643648e+16, 'train_loss': 2.6290146393107476, 'epoch': 5.0})

In [39]:
best_ckpt = trainer.state.best_model_checkpoint
print("Best checkpoint path:", best_ckpt)

Best checkpoint path: output/mt5_small-translationese-mitigation/checkpoint-13375


In [22]:
print(raw_df.sample(5))  # input_text와 target_text가 비정상적으로 동일하거나 빈 값이 있는지 확인


                                              input_text  \
14653            다양한 보고서 양식은 무료일 뿐만 아니라 자유롭게 편집할 수 있습니다.   
74244              이건 당신에게 아주 새로운 음식일 수도 있지만 추천해 드리겠습니다.   
70677                        이 프로젝트에 헌신한 시간과 노력에 감사드립니다.   
5693                         당신과 당신의 친구들에게 초대장을 보내야 합니다!   
94545  "내가 챔피언이 되지 않더라도, 나는 이미 당신의 참석으로 챔피언이 됐다"고 그녀는...   

                                            target_text  
14653    다양한 보고서 양식이 무료 제공될 뿐만 아니라 자유롭게 양식을 편집할 수 있습니다.  
74244                 귀하께는 아주 새로운 음식일지 모르나 추천 드리고 싶습니다.  
70677                         이 프로젝트 진행을 위한 노고에 감사드립니다.  
5693                            여러분과 여러분의 친구들을 초대하겠습니다!  
94545  "챔피언이 되지 못하더라도, 이미 엄마의 참석으로 챔피언이에요"라고 그녀가 말했습니다.  


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_path = "output/mt5_small-translationese-mitigation/checkpoint-13375"  
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to("cuda")

# 예시 문장
input_text = "UV 보호 기능을 가지고 있습니다. 400mm까지의 모든 유해한 파란색 빛을 100% 필터링합니다. 하지만 착용하기에 매우 편리합니다."

# 인퍼런스
inputs = tokenizer(input_text, return_tensors="pt", padding="max_length", truncation=True, max_length=128).to("cuda")
with torch.no_grad():
    outputs = model.generate(**inputs,decoder_start_token_id=tokenizer.eos_token_id, max_new_tokens=256, num_beams=5)
result = tokenizer.decode(outputs[0], skip_special_tokens=True).replace("<extra_id_0>", "").strip()


print("✅ 입력:", input_text)
print("✅ 출력:", result)


✅ 입력: UV 보호 기능을 가지고 있습니다. 400mm까지의 모든 유해한 파란색 빛을 100% 필터링합니다. 하지만 착용하기에 매우 편리합니다.
✅ 출력: UV 보호 기능은 400mm까지의 모든 유해한 파란색 빛을 100% 필터링합니다.


In [41]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# 모델 로딩
model_dir = "output/mt5_small-translationese-mitigation/checkpoint-13375" 
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir).to("cuda")
model.eval()

# 번역투 문장 리스트
# translationese_list = [
#     "그는 나에게 있어서 소중한 존재이다.",
#     "당신이 만약 질문이 있다면, 저에게 말해주세요.",
#     "이 제품은 많은 인기에 의해 판매되고 있습니다.",
#     "이것은 한국에서 만들어진 것입니다.",
#     "이 서비스는 사용자들에게 편리함을 제공하고 있습니다.",
#     "이 책은 매우 흥미로운 내용을 가지고 있습니다.",
#     "나는 어제 서울에 도달하였습니다.",
#     "이 프로그램은 사람들에 의해 널리 사용되고 있습니다.",
#     "그녀는 나에게 있어서 좋은 친구입니다.",
#     "이 영화는 감정적으로 많은 영향을 주었습니다."
# ]
# translationese_list = [
#     "나는 너의 도움에 대해 매우 감사하고 있다.",
#     "그들은 그들의 친구들과 함께 시간을 보내는 것을 즐긴다.",
#     "나는 그것이 나에게 있어서 중요한 결정이었다고 생각한다.",
#     "우리는 그것에 대해 곧 논의할 예정이다.",
#     "그는 회의에 제 시간에 도착하지 못했다.",
#     "나는 그것을 듣게 되어 유감스럽다.",
#     "그녀는 매우 지적인 사람으로 알려져 있다.",
#     "나는 그것이 문제라고 생각하지 않는다.",
#     "우리는 더 나은 결과를 얻기 위해 열심히 일하고 있다.",
#     "그는 내 말에 동의하지 않는 것처럼 보였다.",
#     "그녀는 나에게 그렇게 말하였다.",
#     "나는 이 문제를 해결하기 위해 노력하고 있다.",
#     "그것은 나에게 많은 것을 의미한다.",
#     "그는 그 소식을 듣고 매우 놀랐다.",
#     "우리는 우리가 가진 것에 대해 감사해야 한다."
# ]
# translationese_list = [
#     "나는 그것을 이해하는 것이 어렵다고 여겨진다.",
#     "그녀는 나에게 그것을 설명해 주기를 원했다.",
#     "그 문제는 다양한 방식들로 다루어질 수 있다.",
#     "그들은 그들의 부모님들과 함께 살고 있다.",
#     "우리는 그것을 논의해야 할 필요가 있다.",
#     "이것은 나의 생각으로는 최선의 선택이다.",
#     "그는 나에게 있어서 매우 특별한 사람이다.",
#     "나는 그것에 대해 더 많이 알기를 원한다.",
#     "그녀는 나에게 그녀의 생각들을 공유했다.",
#     "그것은 내가 전에 들어본 적이 없는 이야기이다."
# ]

# 현구와 동일한 input
translationese_list = [
           '철수는 10개의 소를 키운다.',
           '그 아이들은 보육원에 의해 보호되었다.', #보육원은 그 아이들을 보호했다.
           '전통적인 중국 문화에서 결혼에 대한 결정들은 그들의 자녀를 위하는 부모들에 의해 만들어졌다.', #중국의 전통문화로는, 혼인 결정을 자식 대신 부모가 했다.
           '다음 물음에 대해 답하시오.', # 다음 물음에 답하시오.
           '고모는 두 명을 아들을 가지고 있다.', #고모는 아들이 두 명 있다.
           '나는 미국 여행을 가기 위해 영어를 공부했다.', # 나는 미국 여행 가려고 영어를 공부했다.
           '학생들은 공부함에 있어서 불편함이 없었다.', # 학생들은 공부하는 데에 불편함이 없었다.
           '성공하기 위해서는 열심히 노력하지 않으면 안 된다.', # 성공하려면 열심히 노력해야 한다. 
           '그들 대부분은 좋은 직장을 얻는 데는 새로운 아이디어들과 방법들이 있다는 것을 모른다.', #그들 가운데 대다수는 좋은 직업을 구하는 데 있어서 여러 가지 남다른 생각과 방법이 있다는 것을 모른다.
           '두 사람이 주점의 2층에서의 살림을 그만두고 교외로 이사하여 노라의 아기 주디와 살고 있다는 이야기를 들었다.' # 두 사람이 주점의 2층에서 시작한 살림을 그만두고 교외로 이사하여 노라의 아기 주디와 살고 있다는 이야기를 들었다.

]

# 완화된 결과 생성
for idx, sentence in enumerate(translationese_list, 1):
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to("cuda")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            decoder_start_token_id=tokenizer.eos_token_id,
            max_new_tokens=128,
            num_beams=5,
            early_stopping=True
        )
        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

        if "<extra_id_0>" in decoded:
            result = decoded.split("<extra_id_0>", 1)[-1].strip()
        else:
            result = decoded.strip()
        result = result.lstrip(" .>")

    # result = tokenizer.decode(output[0], skip_special_tokens=True).replace("<extra_id_0>", "").strip()

    print(f"입력 {idx}: {sentence}")
    print(f"완화 {idx}: {result}\n")


입력 1: 철수는 10개의 소를 키운다.
완화 1: 철수는 10개의 소를 키운다.

입력 2: 그 아이들은 보육원에 의해 보호되었다.
완화 2: 그 아이들은 보육원에 의해 보호되었습니다.

입력 3: 전통적인 중국 문화에서 결혼에 대한 결정들은 그들의 자녀를 위하는 부모들에 의해 만들어졌다.
완화 3: 전통적인 중국 문화에서 결혼에 대한 결정은 그들의 자녀를 위하는 부모들에 의해 만들어졌습니다.

입력 4: 다음 물음에 대해 답하시오.
완화 4: 다음 물음에 대해 답변해 주십시오.

입력 5: 고모는 두 명을 아들을 가지고 있다.
완화 5: 고모는 두 명을 가지고 있어요.

입력 6: 나는 미국 여행을 가기 위해 영어를 공부했다.
완화 6: 나는 미국 여행을 가기 위해 영어를 공부했습니다.

입력 7: 학생들은 공부함에 있어서 불편함이 없었다.
완화 7: 학생들은 공부함에 있어서 불편함이 없었습니다.

입력 8: 성공하기 위해서는 열심히 노력하지 않으면 안 된다.
완화 8: 성공하기 위해서는 열심히 노력하지 않으면 안 됩니다.

입력 9: 그들 대부분은 좋은 직장을 얻는 데는 새로운 아이디어들과 방법들이 있다는 것을 모른다.
완화 9: 그들 대부분은 좋은 직장을 얻는 데 새로운 아이디어와 방법이 있다는 것을 모른다.

입력 10: 두 사람이 주점의 2층에서의 살림을 그만두고 교외로 이사하여 노라의 아기 주디와 살고 있다는 이야기를 들었다.
완화 10: 두 사람이 주점의 2층에서 살림을 그만두고 교외로 이사하여 노라의 아기 주디와 살고 있다는 이야기를 들었습니다.

